In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.metrics import accuracy_score,f1_score
pd.set_option('display.max_columns', None)

In [19]:
df = pd.read_csv("/Users/benjamincheng/Downloads/cleaned_2019_2024_71.8%_odds.csv", index_col=0)
# use list comphrension to call games from 3/18
january_march_games = df[df['date_next'].str.contains('2024-03') | df['date_next'].str.contains('2024-02')|df['date_next'].str.contains('2024-01')]
january_march_games = january_march_games.reset_index(drop=True)

# Predictors chosen using sequential feature selector trained on ridge classifier
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'calibrated_predictors_ridge_classifier_70_predictors_20.32%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'calibrated_ridge_classifier_70_predictors_20.32%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    clf = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = january_march_games[predictors]
predictions = clf.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([january_march_games['Teams_x'], result], axis=1)
final.columns = ['Teams', 'Predictions']
final['Actual'] = january_march_games['target_x']

accuracy = accuracy_score(final['Actual'], final['Predictions'])
accuracy

0.7005012531328321

In [20]:
probabilities = clf.predict_proba(X)[:, 1]
result = pd.DataFrame({
    'Predictions': predictions,
    'Probabilities': probabilities,  # probabilities of positives
    'Actual': january_march_games['target_x']
}, index=X.index)

# Concatenate the predictions and probabilities with the original data
final = pd.concat([january_march_games['date_next'],january_march_games['Teams_x'], result], axis=1)
final.columns = ['Date','Teams', 'Predictions', 'Probabilities', 'Actual']

# loss = final[final['Predictions']==0]
# loss['Probabilities'] = 1 - loss['Probabilities']
# win = final[final['Predictions']==1]

# complete = pd.concat([win,loss])
# complete = complete.reset_index(drop=True)

In [22]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/risk_management/"
file_name = "calibrated_2024_predictions.csv"
file_path = save_path + file_name
final.to_csv(file_path)